In [1]:
import glob
from asr_evaluation import __main__
import asr_evaluation
import asr.align

In [3]:
transDict = dict()
count = 0
for filename in glob.iglob('/datasets/home/35/335/cs253wgf/291/Data/dev-clean/**/*.trans.txt', recursive=True):
    with open(filename) as f:
        for line in f:
            vals = line.split(" ", 1)
            transDict[vals[0]] = vals[1].lower()
    count += 1
print(count)
print(len(transDict))

97
2703


In [4]:
from __future__ import absolute_import, division, print_function
from timeit import default_timer as timer

import subprocess
import sys
import scipy.io.wavfile as wav
import numpy as np
from deepspeech.model import Model

# These constants control the beam search decoder

# Beam width used in the CTC decoder when building candidate transcriptions
BEAM_WIDTH = 500

# The alpha hyperparameter of the CTC decoder. Language Model weight
LM_WEIGHT = 1.75

# The beta hyperparameter of the CTC decoder. Word insertion weight (penalty)
WORD_COUNT_WEIGHT = 1.00

# Valid word insertion weight. This is used to lessen the word insertion penalty
# when the inserted word is part of the vocabulary
VALID_WORD_COUNT_WEIGHT = 1.00


# These constants are tied to the shape of the graph used (changing them changes
# the geometry of the first layer), so make sure you use the same constants that
# were used during training

# Number of MFCC features to use
N_FEATURES = 26

# Size of the context window used for producing timesteps in the input vector
N_CONTEXT = 9

def convert_samplerate(audio_path):
    sox_cmd = 'sox {} --type raw --bits 16 --channels 1 --rate 16000 - '.format(audio_path)
    try:
        p = subprocess.Popen(sox_cmd.split(),
                             stderr=subprocess.PIPE, stdout=subprocess.PIPE)
        output, err = p.communicate()

        if p.returncode:
            raise RuntimeError('SoX returned non-zero status: {}'.format(err))

    except OSError as e:
        raise OSError('SoX not found, use 16kHz files or install it: ', e)

    audio = np.fromstring(output, dtype=np.int16)
    return 16000, audio

def getTextFromAudio(audio):
    model =  "/datasets/home/35/335/cs253wgf/models/output_graph.pb"
    alphabet = "/datasets/home/35/335/cs253wgf/models/alphabet.txt"
    lm = "/datasets/home/35/335/cs253wgf/models/lm.binary"
    trie = "/datasets/home/35/335/cs253wgf/models/trie"

#     print('Loading model from file %s' % (model), file=sys.stderr)
    model_load_start = timer()
    ds = Model(model, N_FEATURES, N_CONTEXT, alphabet, BEAM_WIDTH)
    model_load_end = timer() - model_load_start
#     print('Loaded model in %0.3fs.' % (model_load_end), file=sys.stderr)

    if lm and trie:
#         print('Loading language model from files %s %s' % (lm, trie), file=sys.stderr)
        lm_load_start = timer()
        ds.enableDecoderWithLM(alphabet, lm, trie, LM_WEIGHT,
                               WORD_COUNT_WEIGHT, VALID_WORD_COUNT_WEIGHT)
        lm_load_end = timer() - lm_load_start
#         print('Loaded language model in %0.3fs.' % (lm_load_end), file=sys.stderr)

    fs , audio1 = wav.read(audio)
    if fs != 16000:
#         if fs < 16000:
#             print('Warning: original sample rate (%d) is lower than 16kHz. Up-sampling might produce erratic speech recognition.' % (fs), file=sys.stderr)
        fs , audio1 = convert_samplerate(audio)
    audio_length = len(audio) * ( 1 / 16000)

#     print('Running inference.', file=sys.stderr)
    inference_start = timer()
    textInferred = ds.stt(audio1, fs);
#     print(textInferred)
    inference_end = timer() - inference_start
#     print('Inference took %0.3fs for %0.3fs audio file.' % (inference_end, audio_length), file=sys.stderr)
    return textInferred

In [5]:
from deepspeech.model import Model

In [ ]:
count = 0
WER = 0
for filename in glob.iglob('/datasets/home/35/335/cs253wgf/291/Data/dev-clean/**/*.wav', recursive=True):
    vals = filename.rsplit("/", 1)
    audioFileName = (vals[1].split(".", 1))[0]
    text1 = transDict[audioFileName]
    text2 = getTextFromAudio(filename)
    if(not text1):
        text1 = ""
    if(not text2):
        text2 = ""
    WER = WER + asr.align.calculate_wer(text1, text2)
    count += 1
print("Done")
print("Total count", count)
print("Average WER:", WER/count)